# You have taken your videos, but what REALLY is going on in your samples?

Rather than manually track how your beads, DNA samples, or active particles are moving, we are going to use a nifty technique called Differential Dynamic Microscopy to exract important the information. 

So, let's begin. Load in all of the python modules in the following cell. (It's ok for the red warning signs to appear...It will still run!)

In [9]:
import numpy as np
import matplotlib
%matplotlib notebook
from matplotlib import pyplot as plt

from IPython.html.widgets import interact, fixed
import ipywidgets as widgets

import io 

font_plt = {'family': 'serif',
            'color':  'darkred',
            'weight': 'normal',
            'size': 10,
            }
font_plt_ax = {'family': 'serif',
               'color':  'black',
               'weight': 'normal',
               'size': 10,
              }
import sys
import glob #glob is helpful for searching for filenames or directories
import ddm_clean as ddm #this is the module containing the differential dynamic microscopy code
import tiff_file
import scipy #scientific python
import pickle #for saving data

In [10]:
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt


Load in the data directory. I like to have a separate folder for just .tif files, to keep things more organized. You can do this or a different system. 

The function glob.glob will pull out files with the specific qualities that you require. Here we want files ending with 'ome.tif' so we add the ** in front of 'ome.tif' to mark that there can be anything before the ome.tif. 

If there are multiple files they will then be sorted into the list named files.

In [16]:
Data_dir = "Z:\\KiraTommy_Summer2019\\Data\\2019-06-11_100nmTEMPOL_1-10_forDDM\\Video_5%\\"
files = glob.glob(Data_dir+"*tif")
files.sort()
print("Found %i files" % len(files))
for f in files: print('\t'+f.split('\\')[-1])


Found 2 files
	Video_NoUV_2_MMStack_Pos0.ome.tif
	Video_NoUV_2_MMStack_Pos0_2x2bin.tif


Next we will set a few parameters and limits for our analysis.

You will have taken videos with upwards of 1000 frames (for DNA videos they are perferably 3000+). These videos can be broken up into groups of different frame differences. We can control what the frame differences are by changing the endframelag variable. 

We have mostly used 998, 1998, and 2998 depending on the number of frames in the video. For more than 3000 frames I tend to use 2998. 

We can also change the number of values sampled out of those different time lags (frame differences). Usually we choose 400, but for a quick overview of how the code work we can stick with 20. 

In [12]:
endframelag = 500
numvaluessampled = 20

dts = ddm.genLogDist(1,endframelag,numvaluessampled)


In [13]:
file_to_analyze = files[1]
print "Analyzing file: %s" % file_to_analyze
im = tiff_file.imread(file_to_analyze)

Analyzing file: Z:\KiraTommy_Summer2019\Data\2019-06-11_100nmTEMPOL_1-10_forDDM\Video_5%\Video_NoUV_2_MMStack_Pos0_2x2bin.tif


In the code block below, we define a mask we can apply to our data. Sometimes this mask helps produce cleaner data at the high spatial frequencies (large qs). 

In [14]:
import scipy
from scipy.signal import blackmanharris as bh #for Blackman-Harris windowing

numPixels = im.shape[1] #determining the number of pixels in an image (just in one dimension, but image should be square.)
x,y = np.meshgrid(bh(numPixels),bh(numPixels)) 
mask = x*y 
im = im * mask

In [15]:
start_frame = 0 #
fft,dtsB = ddm.getFFTDiffsAtTimes(im[start_frame:,:,:], dts, every=None, limitImsTo=None, submean=True)
ravs = ddm.radialAvFFTs_v2(fft)
dts_fft = {'dts':dts, 'ffts':fft, 'ravs':ravs}
pickle.dump(dts_fft, open(file_to_analyze+"_BH_256x256_FFTDIFFS_dts_ravs.p",'wb'))

Once this code has finished running continue on to **Step 2**: DDM fitting of ravs to find taus.